# Setup

In [1]:
%matplotlib inline

In [6]:
import keras
import vgg16
import os
import numpy as np

from vgg16 import Vgg16
from glob import glob

In [3]:
model = Vgg16()

In [25]:
#path = 'data/fish/'
path = 'data/sample_fish/'

# Create Validation Set

In [24]:
for sub_path in os.listdir(path+'/train/'):
    g = glob(path+'train/'+sub_path+'/*.jpg')
    shuf = np.random.permutation(g)
    for i in range(int(len(g)*0.2)):
        os.rename(shuf[i], path+'valid/' + sub_path + '/' + shuf[i].split('/')[-1])

# Fit a Model

In [38]:
bs=64
noe=10

In [51]:
batches = model.get_batches(path+'train', batch_size=bs, shuffle=False)
valid_batches = model.get_batches(path+'valid', batch_size=bs*2)

Found 11 images belonging to 8 classes.
Found 11 images belonging to 8 classes.


In [40]:
model.finetune(batches)

In [41]:
model.fit(batches, valid_batches, nb_epoch=noe)

Epoch 1/10
11/11 [==============================] - 26s - loss: 3.8008 - acc: 0.2727 - val_loss: 2.4274 - val_acc: 0.1818
Epoch 2/10
11/11 [==============================] - 25s - loss: 3.5846 - acc: 0.0000e+00 - val_loss: 1.7090 - val_acc: 0.3636
Epoch 3/10
11/11 [==============================] - 25s - loss: 3.7541 - acc: 0.1818 - val_loss: 1.2029 - val_acc: 0.3636
Epoch 4/10
11/11 [==============================] - 24s - loss: 2.8228 - acc: 0.1818 - val_loss: 0.8084 - val_acc: 0.7273
Epoch 5/10
11/11 [==============================] - 25s - loss: 1.5640 - acc: 0.5455 - val_loss: 0.5237 - val_acc: 0.8182
Epoch 6/10
11/11 [==============================] - 25s - loss: 1.3521 - acc: 0.3636 - val_loss: 0.3417 - val_acc: 0.9091
Epoch 7/10
11/11 [==============================] - 23s - loss: 1.0965 - acc: 0.7273 - val_loss: 0.2203 - val_acc: 0.9091
Epoch 8/10
11/11 [==============================] - 24s - loss: 0.3077 - acc: 0.8182 - val_loss: 0.1364 - val_acc: 0.9091
Epoch 9/10
11/11 [==

In [52]:
batches.filenames

['ALB/img_00003.jpg',
 'ALB/img_00010.jpg',
 'ALB/img_01271.jpg',
 'BET/img_00107.jpg',
 'DOL/img_00734.jpg',
 'LAG/img_00091.jpg',
 'NoF/img_00068.jpg',
 'OTHER/img_00721.jpg',
 'SHARK/img_00033.jpg',
 'SHARK/img_00096.jpg',
 'YFT/img_00641.jpg']

In [53]:
imgs, labels = next(batches)

In [54]:
labels

array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]], dtype=float32)

# Write out Predictions

In [59]:
batches, preds = model.test(path+'test_stg1', batch_size = bs*2)

Found 4 images belonging to 1 classes.


In [60]:
filenames = batches.filenames

with open("sample_sub.csv", "w") as f:
    f.write("image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT\n")
    for pred,fn in zip(preds, filenames):
        pred = ",".join(map(lambda x: str(x), pred))
        f.write(fn+","+pred+"\n")

In [57]:
preds

array([[  9.78287160e-01,   1.17021508e-03,   1.64011959e-03,
          5.07443724e-03,   4.53603640e-03,   7.92800263e-03,
          5.43596456e-04,   8.20445828e-04],
       [  9.85946238e-01,   2.39371031e-04,   7.97419634e-04,
          4.02532278e-05,   3.02053615e-03,   9.68686212e-03,
          1.84084303e-04,   8.52012963e-05],
       [  3.33612028e-04,   2.85495128e-02,   9.90263093e-03,
          1.79892722e-05,   5.80656249e-03,   5.38211584e-01,
          3.36973876e-01,   8.02041888e-02],
       [  9.80785549e-01,   3.48312897e-03,   3.71211441e-04,
          6.60268488e-05,   1.17640225e-02,   1.18196476e-04,
          2.56246794e-03,   8.49252450e-04]], dtype=float32)

In [58]:
batches.filenames

['ALB/img_00003.jpg',
 'ALB/img_00010.jpg',
 'ALB/img_01271.jpg',
 'BET/img_00107.jpg',
 'DOL/img_00734.jpg',
 'LAG/img_00091.jpg',
 'NoF/img_00068.jpg',
 'OTHER/img_00721.jpg',
 'SHARK/img_00033.jpg',
 'SHARK/img_00096.jpg',
 'YFT/img_00641.jpg']